In [600]:
import pandas as pd
import numpy as np

In [601]:
df = pd.read_csv("raw/audio_senp_raw.csv")

In [602]:
def setTokenLength(groupByDoc):
    groupByDoc['token_length'] = groupByDoc.token_id.max()
    return groupByDoc

df = df.groupby('doc_id').apply(setTokenLength)

In [603]:
# calculate dependency_length for each token
for index, row in df.iterrows():
    if row["head_token_id"] == 0:
        df.at[index,'dependency_length'] = 0
    else:
        df.at[index,'dependency_length'] = df.at[index,'token_id'] - df.at[index,'head_token_id']

In [604]:
# calculate jumlah dependency_length for each sentence
current_id = df.loc[0,'doc_id']
sum_dependency_length = 0
df["sum_dependency_length"] = np.NaN

for index, row in df.iterrows():
    if row["doc_id"] == current_id:
        sum_dependency_length += abs(df.at[index,"dependency_length"])
        df.at[index, "sum_dependency_length"] = sum_dependency_length
    else:
        sum_dependency_length = abs(df.at[index,"dependency_length"])
        current_id = row["doc_id"]

In [605]:
#Find Mean Dependency Distance
df["mdd"] = df.sum_dependency_length / (df.token_length - 1)

In [221]:
#bikin algo untuk ubah posisi kata ATAU (PRIO)
#bikin algo untuk ubah posisi root ATAU
#ngitung average dependency dengan nge-random depedendency root nya
#jarak dependensi antar kelas kata, WHERE central node ROOT == VERB




In [626]:
#create a new column new_token_id filled with a randomized order of the token_id

import random

def shuffleTokenId(groupByDoc):
    groupByDoc['new_token_id'] = random.sample(range(1,len(groupByDoc)+1), len(groupByDoc))
    return groupByDoc

In [627]:
def shuffleAndAppend(df, reps):
    appendedDf = pd.DataFrame()
    for i in range(reps):
        df['shuffle_id'] = df['doc_id'] + "-" + str(i)
        appendedDf = appendedDf.append(df.groupby('doc_id').apply(shuffleTokenId))
    return appendedDf

In [628]:
# create a new column prev_rnd_head_token_id filled with a lookup value of the previous head_token_id from rnd_pos_id
def setPrev_head_token_id(group):
    for i, irow in group.iterrows():
        for j, jrow in group.iterrows():
            if irow["new_token_id"] == jrow["token_id"]:
                group.at[i, "prev_head_token_id"] = jrow["head_token_id"]
                break  
    return group

In [629]:
def setNew_head_token_id(group):
    for i, irow in group.iterrows():
        for j, jrow in group.iterrows():
            if irow["prev_head_token_id"] == jrow["new_token_id"]:
                group.at[i, "new_head_token_id"] = jrow["token_id"]
                break
            else:
                group.at[i, "new_head_token_id"] = 0
    return group

In [630]:
# calculate new dependency_length after shuffling
def calculateNewDependencyLength(df):
    for index, row in df.iterrows():
        if row["new_head_token_id"] == 0:
            df.at[index,'new_dependency_length'] = 0
        else:
            df.at[index,'new_dependency_length'] = df.at[index,'token_id'] - df.at[index,'new_head_token_id']
    return df

In [631]:
# calculate jumlah dependency_lenght for each sentence
def calculateNewSumDependencyLength(group):
    group['new_sum_dependency_length'] = np.sum(abs(group['new_dependency_length']))
    return group

In [634]:
def shuffleAndCalculate(df):
    df = df.groupby('doc_id').apply(shuffleTokenId) 
    df = df.groupby('doc_id').apply(setPrev_head_token_id)
    df = df.groupby('doc_id').apply(setNew_head_token_id)
    df = calculateNewDependencyLength(df)
    df = df.groupby('doc_id').apply(calculateNewSumDependencyLength)
    return df

In [660]:
def repeatShuffle(df, reps):
    df_new = pd.DataFrame()
    for i in range(reps):
        temp = shuffleAndCalculate(df)
        temp["shuffle_id"] = str(i)
        temp.doc_id = temp.doc_id + "-s" + temp.shuffle_id 
        df_new = df_new.append(temp)
    
    return df_new

In [ ]:
#85 docs
repeatShuffle(df, 100).to_csv("allshuffle100_out.csv")

In [669]:
len(df.head(997).groupby('doc_id').doc_id.nunique())


85